### Importing Neccesory Libraries

In [7]:
import os
import time
import cv2
import numpy as np
import tensorflow as tf
import pickle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from tensorflow.keras.models import load_model
import mediapipe as mp

### Data Preparation And Visualisations

In [8]:
def load_frames_from_directory(directory):
    img_id=[]
    img_path=[]
    for root, dirs, files in os.walk(directory):
        # temparary image and path
        ti=[]
        tp=[]
        for file in files:
            
            # Check if the file is an image file
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                frame_path = os.path.join(root, file)
                
                if file not in ti:
                    ti.append(file)
                    tp.append(frame_path)
                    
        if ti:
            img_id.append(ti)
            img_path.append(tp)
                    
    return img_id,img_path

All_image_id,All_image_path=load_frames_from_directory('Frames_Sentence_Level')


df1=pd.read_csv('corpus_csv_files\\ISL Corpus sign glosses.csv')
df2=pd.read_excel('corpus_csv_files\\ISL_CSLRT_Corpus_frame_details.xlsx')

data_num=len(All_image_path)

captions = []

for i in range(len(All_image_path)):

    if 'ISL_CSLRT_Corpus\\' + All_image_path[i][0] in df2['Frames path'].values:
        word_value = df2.loc[df2['Frames path'] == 'ISL_CSLRT_Corpus\\' + All_image_path[i][0], 'Sentence'].values
        sign_gloss = df1.loc[df1['Sentence'] == word_value[0], 'SIGN GLOSSES'].values
        if sign_gloss.size<=0:
            sign_gloss=np.array([word_value[0].upper()])
        
    else:
        sign_gloss = np.array(["No caption found"])
        
    captions.append('<start> '+sign_gloss[0]+' <end>')

In [9]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels to numerical values
y = label_encoder.fit_transform(captions)
labels = to_categorical(y).astype(int)

In [15]:
with open('New_class.pkl', 'wb') as file:
    pickle.dump(label_encoder.classes_,file)

In [6]:
sequence=np.load('new_seq.npy')

### Training The Data

In [13]:
X = sequence
y = labels

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
print(y_train.shape)
print(X_train.shape)

(621, 211)
(621, 30, 258)


In [64]:
#del model

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Dropout, Flatten

# Define the model
model = Sequential()

# Add LSTM layer with return_sequences=True for time-distributed behavior
model.add(LSTM(units=256, input_shape=(30, 258), return_sequences=True))
model.add(LSTM(units=256, return_sequences=False))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=98, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 30, 256)           527360    
                                                                 
 lstm_3 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 512)               131584    
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout_3 (Dropout)         (None, 256)              

In [17]:
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

In [18]:
model.fit(X_train, y_train, epochs=1000,validation_split=0.1,callbacks=[early_stopping])

Epoch 1/300
18/18 [==============================] - 14s 418ms/step - loss: 5.3583 - accuracy: 0.0018 - val_loss: 5.3288 - val_accuracy: 0.0000e+00
Epoch 2/300
18/18 [==============================] - 6s 337ms/step - loss: 5.1751 - accuracy: 0.0072 - val_loss: 5.3430 - val_accuracy: 0.0000e+00
Epoch 3/300
18/18 [==============================] - 6s 346ms/step - loss: 5.1271 - accuracy: 0.0072 - val_loss: 5.3125 - val_accuracy: 0.0000e+00
Epoch 4/300
18/18 [==============================] - 7s 371ms/step - loss: 5.0793 - accuracy: 0.0018 - val_loss: 5.3329 - val_accuracy: 0.0000e+00
Epoch 5/300
18/18 [==============================] - 7s 360ms/step - loss: 5.0473 - accuracy: 0.0143 - val_loss: 5.3189 - val_accuracy: 0.0000e+00
Epoch 6/300
18/18 [==============================] - 5s 302ms/step - loss: 5.0326 - accuracy: 0.0108 - val_loss: 5.3566 - val_accuracy: 0.0000e+00
Epoch 7/300
18/18 [==============================] - 6s 330ms/step - loss: 5.0267 - accuracy: 0.0090 - val_loss: 5.44

In [19]:
model.evaluate(X_test,y_test)

5/5 [==============================] - 1s 106ms/step - loss: 19.6131 - accuracy: 0.0064


[19.61314582824707, 0.006410256493836641]

In [20]:
model.save('model03.keras')

In [11]:
model=load_model('model03.keras')